<a href="https://colab.research.google.com/github/mmahyoub/prompt-engineering-workshop/blob/main/Advanced_Prompt_Engineering_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Advanced Prompt Engineering Techniques


In [1]:
# Install packages
!pip install openai langchain langchain_openai langchain_community langchain-chroma -q

In [2]:
from getpass import getpass
api_key = getpass("Enter your OpenAI API Key: ")

Enter your OpenAI API Key: ··········


In [3]:
import os
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
os.environ["OPENAI_API_KEY"] = api_key

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [6]:
from IPython.display import display, Markdown
def display_text(text):
  display(Markdown(text))

#### One-Shot Prompting

> One-shot prompting involves providing a single example to guide the model's response:

In [7]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="""Generate a creative name for a {product} company. For example, a coffee company could be named 'Bean Voyage'.

    Company name: """
)

In [8]:
print("Prompt:")
display_text(prompt.invoke({"product": "coffee"}).text)

# LLM chain
chain = prompt | llm | StrOutputParser()

print("\nOutput:")
display_text(chain.invoke({"product": "coffee"}))

Prompt:


Generate a creative name for a coffee company. For example, a coffee company could be named 'Bean Voyage'.
    
    Company name: 


Output:


Brewed Awakening

#### Few-Shot Prompting

Few-shot prompting uses multiple examples to guide the model.

In [9]:
examples = [
    {"animal": "cat", "description": "A furry feline with retractable claws and independent nature."},
    {"animal": "elephant", "description": "A large mammal with a long trunk and tusks, known for its intelligence."},
    {"animal": "penguin", "description": "A flightless bird adapted to swimming, with a tuxedo-like appearance."}
]

In [10]:
example_prompt = PromptTemplate.from_template("""
Animal: {animal}

Description: {description}
""")

In [11]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Generate a brief description for the given animal:",
    suffix="Animal: {input}",
    input_variables=["input"]
)

In [12]:
display_text(prompt.format(input="tiger"))

Generate a brief description for the given animal:


Animal: cat

Description: A furry feline with retractable claws and independent nature.



Animal: elephant

Description: A large mammal with a long trunk and tusks, known for its intelligence.



Animal: penguin

Description: A flightless bird adapted to swimming, with a tuxedo-like appearance.


Animal: tiger

In [13]:
# Language chain
chain = prompt | llm | StrOutputParser()
display_text(chain.invoke({"input": "tiger"}))

Description: A powerful and majestic big cat with striking orange fur and black stripes, known for its strength and solitary behavior.

#### Chain of Thought (COT)

Chain-of-thought (CoT) prompting generates a sequence of reasoning steps to arrive at a final answer, which is particularly beneficial for complex reasoning tasks in large models. It enhances problem-solving by making the reasoning process explicit, thus improving accuracy in complicated scenarios. For simpler tasks, the benefits of CoT are less pronounced. [Source](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)

In [14]:
cot_template = """Solve the following problem step by step:

Problem: {problem}

Let's approach this step-by-step, use as many steps as required:
1)
2)
3)
...

Therefore, the final answer is:"""

In [15]:
prompt = PromptTemplate(
    template=cot_template,
    input_variables=["problem"]
)

In [16]:
# LLM Chain
chain = prompt | llm | StrOutputParser()

display_text(chain.invoke({"problem": "What is the capital of France?"}))


Let's solve the problem step by step:

1) Identify the country in question: France.
2) Recall the common knowledge or facts about France: it is a country located in Western Europe.
3) Consider the political and administrative aspects of France: every country has a capital city where the government is located.
4) Reflect on historical and cultural knowledge: Paris is widely known as the most significant city in France, often associated with its culture, history, and importance as a capital.
5) Verify the information: Confirm that Paris is recognized as the capital of France in various credible sources, such as geography books, encyclopedias, or authoritative websites.

Therefore, the final answer is: Paris.

In [17]:
display_text(chain.invoke({"problem": "What is the speed of a car if it travels 400 miles per 3 hours."}))

Let's solve the problem step by step:

1) **Identify the given values**: We know the distance the car travels is 400 miles and the time taken is 3 hours.

2) **Recall the formula for speed**: Speed is calculated using the formula:
   \[
   \text{Speed} = \frac{\text{Distance}}{\text{Time}}
   \]

3) **Substitute the values into the formula**: Now we will substitute the distance and time into the speed formula:
   \[
   \text{Speed} = \frac{400 \text{ miles}}{3 \text{ hours}}
   \]

4) **Perform the division**: Now we need to divide 400 by 3:
   \[
   \text{Speed} = 133.33 \text{ miles per hour} \quad (\text{approximately})
   \]

5) **Round the answer if necessary**: Depending on the context, you might round the answer. Here, we can keep it as 133.33 miles per hour, or we could state it as approximately 133 miles per hour if rounded to the nearest whole number.

Therefore, the final answer is:
**The speed of the car is approximately 133.33 miles per hour.**

In [18]:
400/3

133.33333333333334

#### Retrieval Augmented Generation

Retrieval-Augmented Generation (RAG) is a hybrid technique in natural language processing that combines retrieval-based models with generation-based models to improve text generation quality. It works by first retrieving relevant information from a large corpus or knowledge base and then using a generative model, like GPT, to produce text that is more accurate and contextually relevant. This approach enhances the ability of AI to generate fact-based answers or content, making it especially useful in applications like question answering and specialized knowledge tasks, where accuracy and up-to-date information are critical.

In [19]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
# Load, chunk
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [21]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [22]:
# Indexing
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [23]:
vectorstore.similarity_search("What is Zero Shot Prompting?")

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'}, page_content="[My personal spicy take] In my opinion, some prompt engineering papers are not worthy 8 pages long, since those tricks can be explained in one or a few sentences and the rest is all about benchmarking. An easy-to-use and shared benchmark infrastructure should be more beneficial to the community. Iterative prompting or external tool use would not be trivial to set up. Also non-trivial to align the whole research community to adopt it.\nBasic Prompting#\nZero-shot and few-shot learning are two most basic approaches for prompting the model, pioneered by many LLM papers and commonly used for benchmarking LLM performance.\nZero-Shot#\nZero-shot learning is to simply feed the task text to the model and ask for results.\n(All the sentiment analysis examples are from SST-2)\nText: i'll bet the video game is a lot more fun than the film.\nSentiment:\nFew-shot#"),
 Document(metadata=

In [24]:
# Retrieve and generate using the relevant snippets of the blog.
def get_response(query):
  retriever = vectorstore.as_retriever()

  # RAG prompt template
  prompt = hub.pull("rlm/rag-prompt")

  def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)


  rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

  response = rag_chain.invoke(query)
  return response

In [25]:
display_text(get_response("What is Zero Shot Prompting?"))

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Zero Shot Prompting is a method where a model is given a task description without any examples and is asked to produce results based solely on that input. It relies on the model's ability to generalize from its training data to understand and perform the task. This approach is often used in benchmarking the performance of language models.

In [26]:
display_text(get_response("What chain of thought prompting and why it is useful?"))

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Chain-of-thought (CoT) prompting involves generating a series of short sentences that outline reasoning step-by-step, ultimately leading to a final answer. This method is particularly beneficial for complex reasoning tasks, especially when using large language models with over 50 billion parameters, as it clarifies the thought process. For simpler tasks, the advantages of CoT prompting are less significant.